In [2]:
import pandas as pd
import pandera as pa
import numpy as np
from pandera.typing import Series, DataFrame

## What is Pandera?

pandera is a Union.ai open source project that provides a flexible and expressive API for performing data validation on dataframe-like objects to make data processing pipelines more readable and robust.

Dataframes contain information that pandera explicitly validates at runtime. This is useful in production-critical data pipelines or reproducible research settings. With pandera, you can:
- Define a schema once and use it to validate different dataframe types including pandas, polars, dask, modin, and pyspark.pandas.
- Check the types and properties of columns in a pd.DataFrame or values in a pd.Series.
- Perform more complex statistical validation like hypothesis testing.
- Parse data to standardize the preprocessing steps needed to produce valid data.
- Seamlessly integrate with existing data analysis/processing pipelines via function decorators.
- Define dataframe models with the class-based API with pydantic-style syntax and validate dataframes using the typing syntax.
- Synthesize data from schema objects for property-based testing with pandas data structures.
- Lazily Validate dataframes so that all validation rules are executed before raising an error.
- Integrate with a rich ecosystem of python tools like pydantic, fastapi and mypy.


We want to do this on the cleaned data as it makes no sense to do it on raw data as there's likely a whole load of nonsense in there. It's not until you've cleaned the data that you can properly validate the data based on some logic. For example, market values cannot be below 0.

In [2]:
# load cleaned data
# data = pd.read_parquet('../data/player_info_cleaned.parquet')

In [76]:
data = {
    'dob': pd.to_datetime(['1990-05-15', '1988-11-22', '1995-03-10', '1993-07-30', '1992-01-18', '1994-09-05', '1991-12-03', '1989-06-20', '1996-02-14', '1987-08-08']),
    'age': [34, 35, 29, 31, 32, 30, 32, 35, 28, 37],
    'country': ['England', 'Spain', 'Germany', 'France', 'Italy', 'Brazil', 'Argentina', 'Netherlands', 'Portugal', 'England'],
    'current_club': ['Manchester United', 'Chelsea', 'Bayern Munich', 'Paris Saint-Germain', 'Juventus', 'Liverpool', 'Barcelona', 'Ajax', 'Benfica', 'Real Madrid'],
    'height': pd.array([185, 178, None, 176, 188, 182, 170, None, 179, 300], dtype='Int16'),
    'name': ['John Smith', 'Carlos Rodriguez', 'Hans Mueller', 'Pierre Dubois', 'Marco Rossi', 'Felipe Santos', 'Diego Fernandez', 'Jan de Jong', 'Rui Silva', 'Gavin Harris'],
    'position': ['Forward', 'Midfielder', 'Defender', 'Goalkeeper', 'Defender', 'Forward', 'Midfielder', 'Defender', 'Forward', 'Midfielder'],
    'value_euro_m': [75.5, 90.2, 55.8, 40.0, 62.3, 88.7, 70.1, 35.5, 45.9, 95.0],
    'joined_date': pd.to_datetime(['2018-07-01', '2015-08-15', None, '2017-06-30', '2016-09-01', None, '2021-07-15', '2014-08-01', '2022-01-05', '2019-06-01']),
    'number': [9, 10, 4, 1, 3, 11, 8, 5, 7, 17],
    'signed_from': ['Everton', 'Atletico Madrid', 'Borussia Dortmund', None, 'AC Milan', 'Santos', None, 'PSV Eindhoven', 'Sporting CP', 'Newcastle United'],
    'signing_fee_euro_m': [65.0, 80.5, 45.0, None, 55.0, 75.2, 60.8, None, 40.5, 85.0],
    'foot': ['right', 'left', 'right', 'both', 'right', 'left', 'left', 'right', 'both', 'right'],
}

df = pd.DataFrame(data)

In [77]:
df.head()

,dob,age,country,current_club,height,name,position,value_euro_m,joined_date,number,signed_from,signing_fee_euro_m,foot
0,1990-05-15,34,England,Manchester United,185,John Smith,Forward,75.5,2018-07-01,9,Everton,65.0,right
1,1988-11-22,35,Spain,Chelsea,178,Carlos Rodriguez,Midfielder,90.2,2015-08-15,10,Atletico Madrid,80.5,left
2,1995-03-10,29,Germany,Bayern Munich,<NA>,Hans Mueller,Defender,55.8,NaT,4,Borussia Dortmund,45.0,right
3,1993-07-30,31,France,Paris Saint-Germain,176,Pierre Dubois,Goalkeeper,40.0,2017-06-30,1,None,NaN,both
4,1992-01-18,32,Italy,Juventus,188,Marco Rossi,Defender,62.3,2016-09-01,3,AC Milan,55.0,right


## Create Pandera validation schema

There are two methods to validate a dataframe. The first is the DataFrameSchema - an object-based API that allows the specification of schema to validate the index and columns of a pandas dataframe. The second method, and the primary focus of this notebook, is the DataFrameModel which is a class-based API very similar to Pydantic. The DataFrameModel offers direct validation of our dataframes by leveraging both pandera and pandas datatypes. In my opinion, the DataFrameModel is a much more intiuitive way of validating our data, particularly because it integrates seemlessly with pandas. The DataFrameModel can be converted into a DataFrameSchema if necessary so it gives you the best of both worlds.   

In [78]:
df.dtypes

dob                   datetime64[ns]
age                            int64
country                       object
current_club                  object
height                         Int16
name                          object
position                      object
value_euro_m                 float64
joined_date           datetime64[ns]
number                         int64
signed_from                   object
signing_fee_euro_m           float64
foot                          object
dtype: object

In [6]:
# class PlayerSchema(pa.DataFrameModel):
    
#     dob: Series[pd.Timestamp] = pa.Field(nullable=False, ge=pd.Timestamp('1975-01-01'))
#     age: Series[pa.Int64] = pa.Field(ge=0, le=50, nullable=False)
#     country: Series[pa.Category] = pa.Field(nullable=False)
#     current_club: Series[pa.Category] = pa.Field(nullable=True)
#     height: Series[pa.Int16] = pa.Field(ge=120, le=250, nullable=True)
#     name: Series[pa.String] = pa.Field(nullable=False)
#     position: Series[pa.Category] = pa.Field(nullable=False)
#     value_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=200)
#     joined_date: Series[pd.Timestamp] = pa.Field(nullable=True, ge=pd.Timestamp('2000-01-01'))
#     number: Series[pa.Int16] = pa.Field(ge=0, le=99)
#     signed_from: Series[pa.Category] = pa.Field(nullable=True)
#     signing_fee_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=300, nullable=True)
#     tm_id: Series[pa.Int64] = pa.Field(ge=0, unique=False, nullable=False)
#     foot: Series[pa.Category] = pa.Field(nullable=False, isin=['right', 'left', 'both', 'unknown'])
#     season: Series[pa.Int64] = pa.Field(ge=2003, le=2025, nullable=False)
#     team: Series[pa.Category] = pa.Field(nullable=False)
        

In [68]:
class PlayerSchema(pa.DataFrameModel):
    
    dob: Series[pd.Timestamp] = pa.Field(nullable=False, ge=pd.Timestamp('1975-01-01'))
    age: Series[pa.Int64] = pa.Field(ge=0, le=50, nullable=False)
    country: Series[pa.String] = pa.Field(nullable=False)
    current_club: Series[pa.String] = pa.Field(nullable=False)
    height: Series[pa.Int16] = pa.Field(ge=120, le=210, nullable=True)
    name: Series[pa.String] = pa.Field(nullable=False)
    position: Series[pa.String] = pa.Field(nullable=False)
    value_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=200)
    joined_date: Series[pd.Timestamp] = pa.Field(nullable=True, ge=pd.Timestamp('2000-01-01'))
    number: Series[pa.Int64] = pa.Field(ge=0, le=99)
    signed_from: Series[pa.String] = pa.Field(nullable=True)
    signing_fee_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=300, nullable=True)
    foot: Series[pa.String] = pa.Field(nullable=False, isin=['right', 'left', 'both', 'unknown'])

Above, we define a schema by sub-classing pa.DataFrameModel which is done in the same way you sub-class BaseModel in Pydantic. We then populated the schema with the corresponding columns in our dataset, providing the data type expected in each column, and defining boundaries using the pa.Field method. 

In [46]:
@pa.check_types
def load_data() -> DataFrame[PlayerSchema]:
    return df

In [47]:
def validate_data() -> None:
    try:
        stats = load_data()
        print(stats.head())
    except pa.errors.SchemaError as e:
        print(e)

In [48]:
validate_data()

error in check_types decorator of function 'load_data': Column 'height' failed element-wise validator number 1: less_than_or_equal_to(210) failure cases: 300


## Reusing Field objects

In [39]:
from functools import partial

NullableField = partial(pa.Field, nullable=True)
NotNullableField = partial(pa.Field, nullable=False)

In [11]:
# class PlayerSchema(pa.DataFrameModel):
    
#     dob: Series[pd.Timestamp] = pa.Field(nullable=False, ge=pd.Timestamp('1975-01-01'))
#     age: Series[pa.Int64] = pa.Field(ge=0, le=50, nullable=False)
#     country: Series[pa.Category] = NotNullableField()
#     current_club: Series[pa.Category] = NullableField() 
#     height: Series[pa.Int16] = pa.Field(ge=120, le=250, nullable=True)
#     name: Series[pa.String] = NotNullableField()
#     position: Series[pa.Category] = NotNullableField()
#     value_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=200)
#     joined_date: Series[pd.Timestamp] = pa.Field(nullable=True, ge=pd.Timestamp('2000-01-01'))
#     number: Series[pa.Int16] = pa.Field(ge=0, le=99)
#     signed_from: Series[pa.Category] = NullableField()
#     signing_fee_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=300, nullable=True)
#     tm_id: Series[pa.Int64] = pa.Field(ge=0, unique=False, nullable=False)
#     foot: Series[pa.Category] = pa.Field(nullable=False, isin=['Right', 'Left', 'Both'])
#     season: Series[pa.Int64] = pa.Field(ge=2003, le=2025, nullable=False)
#     team: Series[pa.Category] = NotNullableField()

In [69]:
class PlayerSchema(pa.DataFrameModel):
    
    dob: Series[pd.Timestamp] = pa.Field(nullable=False, ge=pd.Timestamp('1975-01-01'))
    age: Series[pa.Int64] = pa.Field(ge=0, le=50, nullable=False)
    country: Series[pa.String] = NotNullableField()
    current_club: Series[pa.String] = NotNullableField()
    height: Series[pa.Int16] = pa.Field(ge=120, le=250, nullable=True)
    name: Series[pa.String] = NotNullableField()
    position: Series[pa.String] = NotNullableField()
    value_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=200)
    joined_date: Series[pd.Timestamp] = pa.Field(nullable=True, ge=pd.Timestamp('2000-01-01'))
    number: Series[pa.Int64] = pa.Field(ge=0, le=99)
    signed_from: Series[pa.String] = NullableField() 
    signing_fee_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=300, nullable=True)
    foot: Series[pa.String] = pa.Field(nullable=False, isin=['right', 'left', 'both', 'unknown'])

## Adding in specific column checks

In [12]:
# class PlayerSchema(pa.DataFrameModel):
    
#     dob: Series[pd.Timestamp] = pa.Field(nullable=False, ge=pd.Timestamp('1975-01-01'))
#     age: Series[pa.Int64] = pa.Field(ge=0, le=50, nullable=False)
#     country: Series[pa.Category] = NotNullableField()
#     current_club: Series[pa.Category] = NullableField() 
#     height: Series[pa.Int16] = pa.Field(ge=120, le=250, nullable=True)
#     name: Series[pa.String] = NotNullableField()
#     position: Series[pa.Category] = NotNullableField()
#     value_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=200)
#     joined_date: Series[pd.Timestamp] = pa.Field(nullable=True, ge=pd.Timestamp('2000-01-01'))
#     number: Series[pa.Int16] = pa.Field(ge=0, le=99)
#     signed_from: Series[pa.Category] = NullableField()
#     signing_fee_euro_m: Series[pa.Float64] = pa.Field(ge=0, le=300, nullable=True)
#     tm_id: Series[pa.Int64] = pa.Field(ge=0, unique=False, nullable=False)
#     foot: Series[pa.Category] = pa.Field(nullable=False, isin=['right', 'left', 'both', 'unknown'])
#     season: Series[pa.Int64] = pa.Field(ge=2003, le=2025, nullable=False)
#     team: Series[pa.Category] = NotNullableField()
        
    # @pa.check(height)
    # def height_check(cls, height: Series[pa.Int16]) -> Series[pa.Int16]:
    #     return height > 120
    

In [50]:
validate_data()

error in check_types decorator of function 'load_data': Column 'height' failed element-wise validator number 1: less_than_or_equal_to(210) failure cases: 300


## Implementing checks in processing step

#### Check heights

In [64]:
def clean_height(df: DataFrame) -> DataFrame[PlayerSchema]:
    data = df.copy()
    data.loc[data[PlayerSchema.height] > 210, PlayerSchema.height] = round(data[PlayerSchema.height].median())
    return data

In [65]:
df = clean_height(df)

In [71]:
@pa.check_types
def validate_data_2(df: DataFrame) -> DataFrame[PlayerSchema]:
    try:
        return df
    except pa.errors.SchemaError as e:
        print(e)

In [59]:
df.dtypes

dob                   datetime64[ns]
age                            int64
country                       object
current_club                  object
height                         Int16
name                          object
position                      object
value_euro_m                 float64
joined_date           datetime64[ns]
number                         int64
signed_from                   object
signing_fee_euro_m           float64
foot                          object
dtype: object

In [72]:
validate_data_2(df)

,dob,age,country,current_club,height,name,position,value_euro_m,joined_date,number,signed_from,signing_fee_euro_m,foot
0,1990-05-15,34,England,Manchester United,185,John Smith,Forward,75.5,2018-07-01,9,Everton,65.0,right
1,1988-11-22,35,Spain,Chelsea,178,Carlos Rodriguez,Midfielder,90.2,2015-08-15,10,Atletico Madrid,80.5,left
2,1995-03-10,29,Germany,Bayern Munich,<NA>,Hans Mueller,Defender,55.8,NaT,4,Borussia Dortmund,45.0,right
3,1993-07-30,31,France,Paris Saint-Germain,176,Pierre Dubois,Goalkeeper,40.0,2017-06-30,1,None,NaN,both
4,1992-01-18,32,Italy,Juventus,188,Marco Rossi,Defender,62.3,2016-09-01,3,AC Milan,55.0,right
5,1994-09-05,30,Brazil,Liverpool,182,Felipe Santos,Forward,88.7,NaT,11,Santos,75.2,left
6,1991-12-03,32,Argentina,Barcelona,170,Diego Fernandez,Midfielder,70.1,2021-07-15,8,None,60.8,left
7,1989-06-20,35,Netherlands,Ajax,<NA>,Jan de Jong,Defender,35.5,2014-08-01,5,PSV Eindhoven,NaN,right
8,1996-02-14,28,Portugal,Benfica,179,Rui Silva,Forward,45.9,2022-01-05,7,Sporting CP,40.5,both
9,1987-08-08,37,England,Real Madrid,180,Gavin Harris,Midfielder,95.0,2019-06-01,17,Newcastle United,85.0,right


In [74]:
# validate dataframe
DataFrame[PlayerSchema](df)

,dob,age,country,current_club,height,name,position,value_euro_m,joined_date,number,signed_from,signing_fee_euro_m,foot
0,1990-05-15,34,England,Manchester United,185,John Smith,Forward,75.5,2018-07-01,9,Everton,65.0,right
1,1988-11-22,35,Spain,Chelsea,178,Carlos Rodriguez,Midfielder,90.2,2015-08-15,10,Atletico Madrid,80.5,left
2,1995-03-10,29,Germany,Bayern Munich,<NA>,Hans Mueller,Defender,55.8,NaT,4,Borussia Dortmund,45.0,right
3,1993-07-30,31,France,Paris Saint-Germain,176,Pierre Dubois,Goalkeeper,40.0,2017-06-30,1,None,NaN,both
4,1992-01-18,32,Italy,Juventus,188,Marco Rossi,Defender,62.3,2016-09-01,3,AC Milan,55.0,right
5,1994-09-05,30,Brazil,Liverpool,182,Felipe Santos,Forward,88.7,NaT,11,Santos,75.2,left
6,1991-12-03,32,Argentina,Barcelona,170,Diego Fernandez,Midfielder,70.1,2021-07-15,8,None,60.8,left
7,1989-06-20,35,Netherlands,Ajax,<NA>,Jan de Jong,Defender,35.5,2014-08-01,5,PSV Eindhoven,NaN,right
8,1996-02-14,28,Portugal,Benfica,179,Rui Silva,Forward,45.9,2022-01-05,7,Sporting CP,40.5,both
9,1987-08-08,37,England,Real Madrid,180,Gavin Harris,Midfielder,95.0,2019-06-01,17,Newcastle United,85.0,right
